In [1]:
#Setting up the code and installing required packages 
from util import *
from glob import glob
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.geometry import LineString, Polygon
pd.set_option("display.max_columns", None)

In [2]:
#Navigate to intersects shapefile and separates out the year, finds how many years from 1800 and how many years to 2100
gdf = gpd.read_file(f"Data/Merged Intersects_UniqueID/JackettIsland_Intersects.shp")
gdf["Date"] = pd.to_datetime(gdf.ShorelineI, dayfirst=True, format='mixed')
gdf["Year"] = gdf.Date.dt.year
gdf["YearsSinceBase"] = (gdf.Date - pd.Timestamp(1800, 1, 1)).dt.days / 365.25
gdf["YearsUntilFuture"] = (
    pd.Timestamp(2100, 1, 1) - gdf.Date
    ).dt.days / 365.25
gdf.Date = gdf.Date.astype(str)
gdf

,ShorelineI,BaselineID,Distance,IntersectX,IntersectY,Uncertaint,Unique_ID,Date,geometry,Year,YearsSinceBase,YearsUntilFuture
0,04/19/2017,1.0,-97.80,1.602805e+06,5.444480e+06,2.94,2.010001e+11,2017-04-19,POINT Z (1602805.403 5444479.642 0.000),2017,217.292266,82.702259
1,01/13/2000,1.0,-54.62,1.602842e+06,5.444502e+06,5.32,2.010001e+11,2000-01-13,POINT Z (1602842.047 5444502.478 0.000),2000,200.027379,99.967146
2,02/19/2010,1.0,-69.27,1.602830e+06,5.444495e+06,2.32,2.010001e+11,2010-02-19,POINT Z (1602829.612 5444494.729 0.000),2010,210.130048,89.864476
3,09/13/1985,1.0,-54.69,1.602842e+06,5.444502e+06,4.33,2.010001e+11,1985-09-13,POINT Z (1602841.988 5444502.442 0.000),1985,185.694730,114.299795
4,02/04/1967,1.0,-65.04,1.602833e+06,5.444497e+06,5.94,2.010001e+11,1967-04-02,POINT Z (1602833.209 5444496.970 0.000),1967,167.244353,132.750171
...,...,...,...,...,...,...,...,...,...,...,...,...
2337,02/22/1940,3.0,-29.51,1.603246e+06,5.444123e+06,3.77,2.010002e+11,1940-02-22,POINT Z (1603246.456 5444123.385 0.000),1940,140.136893,159.857632
2338,06/25/2013,3.0,-113.28,1.603204e+06,5.444051e+06,2.30,2.010002e+11,2013-06-25,POINT Z (1603203.881 5444051.243 0.000),2013,213.475702,86.518823
2339,08/28/2006,3.0,-109.38,1.603206e+06,5.444055e+06,2.32,2.010002e+11,2006-08-28,POINT Z (1603205.860 5444054.597 0.000),2006,206.650240,93.344285
2340,01/31/1980,3.0,-115.24,1.603203e+06,5.444050e+06,5.31,2.010002e+11,1980-01-31,POINT Z (1603202.885 5444049.556 0.000),1980,180.076660,119.917864


In [6]:
#Navigates to the rates shapefile
lines = gpd.read_file("Data/Merged Rates_UniqueID/JackettIsland_Rates_UniqueID.shp")
lines

,ShrCount,LRR,LR2,LCI,LSE,EPR,EPRunc,NSM,SCE,WLR,WR2,WCI,WSE,TCD,Region,MEAS,RegCode,DIST,Start_date,End_date,Duration,Unique_ID,geometry
0,13.0,-0.16,0.07,0.32,16.77,-0.07,0.06,-5.38,44.09,-0.19,0.08,0.34,4.94,7410.848139,Tasman,1474.414593,201,147441,1940-02-22,2020-07-01,80,2.010001e+11,POINT Z (1602827.257 5444493.261 0.000)
1,13.0,-0.22,0.13,0.30,15.96,-0.12,0.06,-9.85,44.06,-0.25,0.15,0.32,4.70,7420.848139,Tasman,1464.391639,201,146439,1940-02-22,2020-07-01,80,2.010001e+11,POINT Z (1602821.392 5444501.389 0.000)
2,13.0,-0.26,0.19,0.28,15.15,-0.18,0.06,-14.58,44.19,-0.29,0.21,0.30,4.41,7430.848139,Tasman,1454.361268,201,145436,1940-02-22,2020-07-01,80,2.010001e+11,POINT Z (1602815.442 5444509.464 0.000)
3,13.0,-0.30,0.26,0.27,14.45,-0.24,0.06,-19.54,44.86,-0.34,0.29,0.28,4.13,7440.848139,Tasman,1444.330360,201,144433,1940-02-22,2020-07-01,80,2.010001e+11,POINT Z (1602809.485 5444517.535 0.000)
4,13.0,-0.34,0.33,0.26,14.09,-0.33,0.06,-26.59,45.63,-0.39,0.37,0.27,4.01,7450.848139,Tasman,1434.299451,201,143429,1940-02-22,2020-07-01,80,2.010001e+11,POINT Z (1602803.528 5444525.605 0.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,11.0,-0.99,0.82,0.28,13.37,-1.30,0.06,-100.43,100.43,-0.99,0.87,0.23,3.15,13980.696530,Tasman,2071.156545,201,207115,1940-02-22,2017-04-19,77,2.010002e+11,POINT Z (1603231.181 5444057.080 0.000)
179,11.0,-1.01,0.84,0.27,12.81,-1.33,0.06,-102.67,102.67,-1.01,0.88,0.22,3.06,13990.696530,Tasman,2080.690439,201,208069,1940-02-22,2017-04-19,77,2.010002e+11,POINT Z (1603238.705 5444051.224 0.000)
180,11.0,-0.99,0.83,0.28,13.26,-1.31,0.06,-101.16,101.16,-0.99,0.87,0.23,3.14,14000.696530,Tasman,2070.605645,201,207060,1940-02-22,2017-04-19,77,2.010002e+11,POINT Z (1603230.754 5444057.428 0.000)
181,11.0,-0.95,0.80,0.29,13.66,-1.30,0.06,-100.62,100.62,-0.95,0.85,0.24,3.27,14010.696530,Tasman,2060.477921,201,206047,1940-02-22,2017-04-19,77,2.010002e+11,POINT Z (1603222.954 5444063.887 0.000)


In [7]:
#Finding the azimuth of the transect lines in the rates shapefile and addding it as an attribute
def get_azimuth(line):
    p1, p2 = line.coords
    azimuth = math.degrees(math.atan2(p2[0]-p1[0], p2[1]-p1[1]))
    if azimuth < 0:
        azimuth += 360
    return azimuth
lines["Azimuth"] = lines.geometry.apply(get_azimuth)
lines.to_file("Data/Merged Rates_UniqueID/JackettIsland_Rates_UniqueID.shp")
lines

ValueError: not enough values to unpack (expected 2, got 1)

In [7]:
#Function allows code to find the necessary information (Azimuth, how many intersect points along each transect) to run projection
def get_transect_metadata(transect_lines_shapefile: str):
    lines = gpd.read_file(transect_lines_shapefile)
    if "TransectID" in lines.columns:
        lines = lines.set_index("TransectID").sort_index()
    else:
        lines = lines.set_index("TransOrder").sort_index()
    lines["dist_to_neighbour"] = lines.distance(lines.shift(-1))
    breakpoints = lines.dist_to_neighbour[lines.dist_to_neighbour > 105]
    lines["group"] = pd.Series(range(len(breakpoints)), index=breakpoints.index)
    lines["group"] = lines.group.bfill().fillna(len(breakpoints)).astype(int)
    metadata = lines[["Azimuth", "group"]].to_dict(orient="index")
    return metadata

transect_metadata = get_transect_metadata(f"TEST/CosyNook_rates.shp")
transect_metadata

{1.0: {'Azimuth': 44.99999905663656, 'group': 0},
 2.0: {'Azimuth': 44.99999905663656, 'group': 0},
 3.0: {'Azimuth': 32.106256486836365, 'group': 0},
 4.0: {'Azimuth': 32.106256486836365, 'group': 0},
 5.0: {'Azimuth': 32.106256132396354, 'group': 0},
 6.0: {'Azimuth': 32.106256486836365, 'group': 0},
 7.0: {'Azimuth': 46.0050867002439, 'group': 0},
 8.0: {'Azimuth': 46.0050867002439, 'group': 0},
 9.0: {'Azimuth': 46.0050867002439, 'group': 0},
 10.0: {'Azimuth': 53.80679398121416, 'group': 0},
 11.0: {'Azimuth': 53.80679398121416, 'group': 0},
 12.0: {'Azimuth': 53.80679398121416, 'group': 0},
 13.0: {'Azimuth': 50.52753927360015, 'group': 0},
 14.0: {'Azimuth': 50.52753927360015, 'group': 0},
 15.0: {'Azimuth': 50.52753927360015, 'group': 0},
 16.0: {'Azimuth': 49.08561762017961, 'group': 0},
 17.0: {'Azimuth': 49.08561587292389, 'group': 0},
 18.0: {'Azimuth': 37.34934837495274, 'group': 0},
 19.0: {'Azimuth': 37.34934837495274, 'group': 0},
 20.0: {'Azimuth': 37.34934837495274, '

In [8]:
#Linear regression is run here. See util.py for the breakdown on linear_models
linear_models = fit(gdf, transect_metadata)
linear_models

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse
0,1.0,-0.050725,-21.281314,0,0.234940,1.874489,5.495887,2.344331
1,2.0,-0.007674,-30.220330,0,0.009368,1.447134,4.084100,2.020916
2,3.0,0.017995,-35.889901,0,0.028782,1.934398,7.167354,2.677191
3,4.0,0.017791,-36.180259,0,0.024229,2.255094,8.360880,2.891519
4,5.0,0.020190,-37.052797,0,0.025692,2.507454,10.139300,3.184227
...,...,...,...,...,...,...,...,...
123,124.0,-0.001832,-27.659415,0,0.002722,0.692081,0.806731,0.898182
124,125.0,0.013290,-31.737070,0,0.054908,1.025165,1.994196,1.412160
125,126.0,0.114650,-50.415902,0,0.518132,2.347711,8.018416,2.831681
126,127.0,0.064432,-42.418093,0,0.238757,2.181754,8.681960,2.946517


In [64]:
#Only run if rolling average is needed. Otherwise SKIP THIS
#linear_models = fit(gdf, transect_metadata)
#rolled_slopes = linear_models.groupby("group").slope.rolling(10, min_periods=1).mean().dropna().reset_index(level=0)
#linear_models.slope = rolled_slopes.slope
#linear_models.dropna(inplace=True)
#linear_models

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse
0,1.0,1.137493,-287.113293,0,0.899979,7.719117,81.226923,9.012598
1,2.0,1.116044,-271.071182,0,0.918585,7.778484,86.694893,9.311009
2,3.0,1.126207,-276.724950,0,0.950353,5.837518,56.063658,7.487567
3,4.0,1.130674,-276.507210,0,0.932981,5.839418,70.903340,8.420412
4,5.0,1.127845,-269.600165,0,0.954922,5.013590,44.378314,6.661705
...,...,...,...,...,...,...,...,...
100,101.0,0.164541,-61.579520,0,0.243565,6.791799,64.534554,8.033340
101,102.0,0.170545,-63.207372,0,0.259564,6.996329,67.737057,8.230253
102,103.0,0.177131,-69.212941,0,0.240615,8.333542,93.865278,9.688409
103,104.0,0.181155,-65.105831,0,0.239869,7.921248,80.745771,8.985865


In [10]:
#Coordinates of the projected shoreline are plotted here

#Changed coordinate function by making old_x and old_y negative 
def calculate_new_coordinates(old_x, old_y, bearing, distance):
    bearing_radians = math.radians(bearing)
    new_x = old_x + (distance * math.sin(bearing_radians))
    new_y = old_y + (distance * math.cos(bearing_radians))
    point = Point(new_x, new_y)
    assert not point.is_empty
    return point

#Removed other model equations and changed Azimuth addtion from 180 to 360 deg
def predict(
    df: pd.DataFrame,
    linear_models: pd.DataFrame,
    transect_metadata: dict,
):
    """_summary_

    Args:
        df (pd.DataFrame): dataframe with columns: TransectID, Date, Distance, YearsSinceBase
        linear_models (pd.DataFrame): dataframe with columns: TransectID, slope, intercept
        transect_metadata (dict): dict lookup of TransectID to Azimuth & group
        
    Returns:
        pd.DataFrame: resulting prediction points for the year 2100
    """
    results = []
    for i, row in linear_models.iterrows():
        transect_ID = row.TransectID
        transect_df = df[df.TransectID == transect_ID]
        latest_row = transect_df[transect_df.Date == transect_df["Date"].max()].iloc[0]
        future_year = int(row.get("FUTURE_YEAR", FUTURE_YEAR))
        result = row.to_dict()
        result.update({
            "TransectID": transect_ID,
            "BaselineID": latest_row.BaselineID,
            "group": row.group,
            "Year": future_year,
            "ocean_point": calculate_new_coordinates(
                latest_row.geometry.x,
                latest_row.geometry.y,
                transect_metadata[transect_ID]["Azimuth"] + 180,
                500,
            ),
        })
        
        model = "linear"
        slope = row.slope
        intercept = row.intercept

        predicted_distance = slope * (future_year - 1800) + intercept
        distance_difference = latest_row.Distance - predicted_distance
        result[f"{model}_model_point"] = calculate_new_coordinates(
            latest_row.geometry.x,
            latest_row.geometry.y,
            transect_metadata[transect_ID]["Azimuth"],
            distance_difference,
        )
        result[f"{model}_model_predicted_distance"] = predicted_distance
        result[f"{model}_model_distance"] = distance_difference
        results.append(result)
    results = gpd.GeoDataFrame(results)
    return results

In [11]:
#Projection file is created here with the stats and coordinate points in table format
results = predict(gdf, linear_models, transect_metadata)
results

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse,BaselineID,Year,ocean_point,linear_model_point,linear_model_predicted_distance,linear_model_distance
0,1.0,-0.050725,-21.281314,0.0,0.234940,1.874489,5.495887,2.344331,0.0,2100,POINT (1193577.077097776 4855106.457222441),POINT (1193932.2843183058 4855461.664454667),-36.498877,2.338877
1,2.0,-0.007674,-30.220330,0.0,0.009368,1.447134,4.084100,2.020916,0.0,2100,POINT (1193569.5301557889 4855113.052416062),POINT (1193922.3993344835 4855465.921606376),-32.522386,-0.967614
2,3.0,0.017995,-35.889901,0.0,0.028782,1.934398,7.167354,2.677191,0.0,2100,POINT (1193644.2343890858 4855054.661997621),POINT (1193908.327708691 4855475.560717066),-30.491350,-3.108650
3,4.0,0.017791,-36.180259,0.0,0.024229,2.255094,8.360880,2.891519,0.0,2100,POINT (1193636.0514251348 4855060.435390343),POINT (1193900.0446379222 4855481.174564551),-30.842999,-3.297001
4,5.0,0.020190,-37.052797,0.0,0.025692,2.507454,10.139300,3.184227,0.0,2100,POINT (1193627.9255033901 4855066.299688667),POINT (1193891.654518527 4855486.617803653),-30.995916,-3.794084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,124.0,-0.001832,-27.659415,0.0,0.002722,0.692081,0.806731,0.898182,7.0,2100,POINT (1193360.3988743809 4855435.913612373),POINT (1192859.779767167 4855435.913612373),-28.209107,0.619107
124,125.0,0.013290,-31.737070,0.0,0.054908,1.025165,1.994196,1.412160,7.0,2100,POINT (1193360.1571609178 4855425.913612373),POINT (1192860.237233766 4855425.913612373),-27.749927,-0.080073
125,126.0,0.114650,-50.415902,0.0,0.518132,2.347711,8.018416,2.831681,7.0,2100,POINT (1193361.61954932 4855477.617708033),POINT (1192873.2356566475 4855413.9154609125),-16.020865,-7.479135
126,127.0,0.064432,-42.418093,0.0,0.238757,2.181754,8.681960,2.946517,7.0,2100,POINT (1193362.5000820463 4855422.330449378),POINT (1192866.9260606398 4855405.241690217),-23.088568,-4.131432


In [12]:
#Spatial reference added to the results
results.set_geometry("linear_model_point", inplace=True, crs=2193)
results

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse,BaselineID,Year,ocean_point,linear_model_point,linear_model_predicted_distance,linear_model_distance
0,1.0,-0.050725,-21.281314,0.0,0.234940,1.874489,5.495887,2.344331,0.0,2100,POINT (1193577.077097776 4855106.457222441),POINT (1193932.284 4855461.664),-36.498877,2.338877
1,2.0,-0.007674,-30.220330,0.0,0.009368,1.447134,4.084100,2.020916,0.0,2100,POINT (1193569.5301557889 4855113.052416062),POINT (1193922.399 4855465.922),-32.522386,-0.967614
2,3.0,0.017995,-35.889901,0.0,0.028782,1.934398,7.167354,2.677191,0.0,2100,POINT (1193644.2343890858 4855054.661997621),POINT (1193908.328 4855475.561),-30.491350,-3.108650
3,4.0,0.017791,-36.180259,0.0,0.024229,2.255094,8.360880,2.891519,0.0,2100,POINT (1193636.0514251348 4855060.435390343),POINT (1193900.045 4855481.175),-30.842999,-3.297001
4,5.0,0.020190,-37.052797,0.0,0.025692,2.507454,10.139300,3.184227,0.0,2100,POINT (1193627.9255033901 4855066.299688667),POINT (1193891.655 4855486.618),-30.995916,-3.794084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,124.0,-0.001832,-27.659415,0.0,0.002722,0.692081,0.806731,0.898182,7.0,2100,POINT (1193360.3988743809 4855435.913612373),POINT (1192859.780 4855435.914),-28.209107,0.619107
124,125.0,0.013290,-31.737070,0.0,0.054908,1.025165,1.994196,1.412160,7.0,2100,POINT (1193360.1571609178 4855425.913612373),POINT (1192860.237 4855425.914),-27.749927,-0.080073
125,126.0,0.114650,-50.415902,0.0,0.518132,2.347711,8.018416,2.831681,7.0,2100,POINT (1193361.61954932 4855477.617708033),POINT (1192873.236 4855413.915),-16.020865,-7.479135
126,127.0,0.064432,-42.418093,0.0,0.238757,2.181754,8.681960,2.946517,7.0,2100,POINT (1193362.5000820463 4855422.330449378),POINT (1192866.926 4855405.242),-23.088568,-4.131432


In [13]:
#Line and polygon shapefiles are created here 
def prediction_results_to_line_polygon(results: gpd.GeoDataFrame):
    lines = []
    polygons = []
    for group_name, group_data in results.groupby(["BaselineID", "group"]):
        if len(group_data) > 1:
            # Convert the points to LineString
            line = LineString(list(group_data.geometry))
            lines.append(line)
            # Convert the points to a closed Polygon
            polygon = Polygon(list(group_data.geometry) + list(group_data.ocean_point)[::-1])
            polygons.append(polygon)
    lines = gpd.GeoSeries(lines, crs=2193)
    polygons = gpd.GeoSeries(polygons, crs=2193)
    return lines, polygons

In [14]:
lines, poly = prediction_results_to_line_polygon(results)

In [45]:
#Saving line and polygon projection file to Z drive. Change file location accordingly  
lines, poly = prediction_results_to_line_polygon(results)
lines.to_file("Z:\DSAS\Region\Southland\BigBay\BigBay_projection_output_lines.shp")
poly.to_file("Z:\DSAS\Region\Southland\BigBay\BigBay_projection_output_polygon.shp")

In [46]:
#Saving line and polygon projection file to folder in VS Code. Change file location accordingly
lines, poly = prediction_results_to_line_polygon(results)
lines.to_file("TEST\Southland_BigBay\BigBay_projection_output_lines.shp")
poly.to_file("TEST\Southland_BigBay\BigBay_projection_output_polygon.shp")

In [15]:
#Quick visualisation of projected polygon and historic shorelines 
m = poly.explore(tiles="Esri.WorldImagery")
gpd.GeoDataFrame(results.drop(columns=["ocean_point", "linear_model_point"]), geometry=results.linear_model_point).explore(m=m)
gdf.explore("Year", legend=True, m=m)